In [2]:
!pip install gym

In [0]:
# Add necessary headers
import gym
import numpy as np
from collections import deque
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import adam
from keras.activations import relu, linear

In [0]:
# Render the environment 
env = gym.make('LunarLander-v2')
env.seed(0)
np.random.seed(0)

In [0]:
class DeepQNetwork:
  # A class to implement the deep q-learning algorithm

  def __init__(self, state_space, action_space, alpha=0.001, gamma=0.99, epsilon=1.0):
    self.state_space = state_space
    self.action_space = action_space
    self.alpha = alpha
    self.gamma = gamma
    self.epsilon = epsilon
    self.epsilon_decay = 0.99
    self.epsilon_min = 0.01
    self.batch_size = 50
    self.memory = deque(maxlen=1000000)

  def create_model(self):
    model = Sequential()
    model = Sequential()
    model.add(Dense(150, input_dim=self.state_space, activation=relu))
    model.add(Dense(120, activation=relu))
    model.add(Dense(self.action_space, activation=linear))
    model.compile(loss='mse', optimizer=adam(lr=self.alpha))
    return model
